In [2]:
# 구글 드라이브 마운트
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
# 라이브러리 import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
# 데이터 불러오기, gameId column 제거
df_all = pd.read_csv('/content/gdrive/My Drive/data/high_diamond_ranked_10min.csv')
df_all = df_all.drop(['gameId'], axis='columns')

df_data = df_all.iloc[:, 1:]
df_label = df_all.iloc[:, 0]

print(df_all.columns)
print(df_data.shape)
print(df_label.shape)

Index(['blueWins', 'blueWardsPlaced', 'blueWardsDestroyed', 'blueFirstBlood',
       'blueKills', 'blueDeaths', 'blueAssists', 'blueEliteMonsters',
       'blueDragons', 'blueHeralds', 'blueTowersDestroyed', 'blueTotalGold',
       'blueAvgLevel', 'blueTotalExperience', 'blueTotalMinionsKilled',
       'blueTotalJungleMinionsKilled', 'blueGoldDiff', 'blueExperienceDiff',
       'blueCSPerMin', 'blueGoldPerMin', 'redWardsPlaced', 'redWardsDestroyed',
       'redFirstBlood', 'redKills', 'redDeaths', 'redAssists',
       'redEliteMonsters', 'redDragons', 'redHeralds', 'redTowersDestroyed',
       'redTotalGold', 'redAvgLevel', 'redTotalExperience',
       'redTotalMinionsKilled', 'redTotalJungleMinionsKilled', 'redGoldDiff',
       'redExperienceDiff', 'redCSPerMin', 'redGoldPerMin'],
      dtype='object')
(9879, 38)
(9879,)


In [5]:
# train set, test set으로 분할
train_size = int(df_all.shape[0] * 0.8)

X_train = df_all.iloc[:train_size, 1:]
X_test = df_all.iloc[train_size:, 1:]

Y_train = df_all.iloc[:train_size, 0]
Y_test = df_all.iloc[train_size:, 0]

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(7903, 38)
(1976, 38)
(7903,)
(1976,)


In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(38, input_dim=38, activation='relu'))
model.add(Dense(38, activation='relu'))
model.add(Dense(38, activation='relu'))
model.add(Dense(38, activation='relu'))
model.add(Dense(38, activation='relu'))
model.add(Dense(38, activation='relu'))
model.add(Dense(38, activation='relu'))
model.add(Dense(38, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 38)                1482      
_________________________________________________________________
dense_1 (Dense)              (None, 38)                1482      
_________________________________________________________________
dense_2 (Dense)              (None, 38)                1482      
_________________________________________________________________
dense_3 (Dense)              (None, 38)                1482      
_________________________________________________________________
dense_4 (Dense)              (None, 38)                1482      
_________________________________________________________________
dense_5 (Dense)              (None, 38)                1482      
_________________________________________________________________
dense_6 (Dense)              (None, 38)                1

In [7]:
from keras.callbacks import EarlyStopping

# Early Stopping을 위한 callback
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10)
model.fit(X_train, Y_train, epochs=200, batch_size=10, callbacks=[es])

Epoch 1/200
791/791 [==============================] - 1s 2ms/step - loss: 3.6539 - accuracy: 0.6471
Epoch 2/200
791/791 [==============================] - 1s 2ms/step - loss: 0.9859 - accuracy: 0.6533
Epoch 3/200
791/791 [==============================] - 1s 2ms/step - loss: 1.0038 - accuracy: 0.6603
Epoch 4/200
791/791 [==============================] - 1s 2ms/step - loss: 1.0113 - accuracy: 0.6554
Epoch 5/200
791/791 [==============================] - 1s 2ms/step - loss: 0.8937 - accuracy: 0.6551
Epoch 6/200
791/791 [==============================] - 1s 2ms/step - loss: 0.9184 - accuracy: 0.6604
Epoch 7/200
791/791 [==============================] - 1s 2ms/step - loss: 0.8035 - accuracy: 0.6615
Epoch 8/200
791/791 [==============================] - 1s 2ms/step - loss: 0.8437 - accuracy: 0.6586
Epoch 9/200
791/791 [==============================] - 1s 2ms/step - loss: 0.7940 - accuracy: 0.6618
Epoch 10/200
791/791 [==============================] - 1s 2ms/step - loss: 0.7401 - accura

In [8]:
_, accuracy = model.evaluate(X_test, Y_test)
print('Accuracy: %.2f' % (accuracy * 100))

62/62 [==============================] - 0s 924us/step - loss: 0.5523 - accuracy: 0.7227
Accuracy: 72.27


In [9]:
# scikit-learn의 패키지를 이용해서 accuracy, precision, recall, f1-score 구하기
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
prediction = model.predict(X_test) > 0.5

acc = 100 * accuracy_score(Y_test, prediction)
pre = 100 * precision_score(Y_test, prediction)
rec = 100 * recall_score(Y_test,prediction)
f1 = 100 * f1_score(Y_test,prediction)

print('accuracy : %.2f%%'%(acc))
print('Precision: %.2f%%\nRecall: %.2f%%\nF1-score: %.2f%% '%(pre, rec, f1))

accuracy : 72.27%
Precision: 69.96%
Recall: 78.82%
F1-score: 74.13% 
